<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/Kripto_Dusen_Yukselen_Trendi_K%C4%B1ranlar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Kütüphanelerin İndirilmesi
!pip install git+https://github.com/rongardF/tvdatafeed tradingview-screener
!pip install git+https://github.com/rongardF/tvdatafeed tradingview-screener matplotlib scipy
!pip install tqdm  # Progress bar için tqdm kütüphanesi

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
from tqdm import tqdm  # Progress bar için tqdm

# Kullanıcının zaman dilimi sayısını seçmesini sağlayan fonksiyon
def get_interval_count_choice():

    print("Kaç farklı zaman diliminde tarama yapmak istiyorsunuz?")
    print("1: Tek Zaman Dilimi")
    print("2: İki Zaman Dilimi")

    choice = input("Seçiminizi yapın (1-2): ")

    if choice == '1':
        return 1
    elif choice == '2':
        return 2
    else:
        print("Geçersiz seçim, varsayılan olarak iki zaman dilimi seçildi.")
        return 2

# Kullanıcının zaman dilimini seçmesini sağlayan fonksiyon
def get_interval_choice(count=2):

    intervals = {
        '1': ('15 Dakika', Interval.in_15_minute, 144),  # 15 Dakika -> 144 bar
        '2': ('30 Dakika', Interval.in_30_minute, 89),  # 30 Dakika -> 89 bar
        '3': ('1 Saat', Interval.in_1_hour, 89),        # 1 Saat -> 89 bar
        '4': ('4 Saat', Interval.in_4_hour, 89),        # 4 Saat -> 89 bar
        '5': ('Günlük', Interval.in_daily, 55),         # Günlük -> 55 bar
        '6': ('Haftalık', Interval.in_weekly, 55),      # Haftalık -> 55 bar
    }
    print("Lütfen zaman dilimi(lerini) seçin:")
    for key, value in intervals.items():
        print(f"{key}: {value[0]} - {value[2]} bar")

    choice_1 = input("Birinci zaman dilimini seçin (1-6): ")
    interval_1 = intervals.get(choice_1, ('4 Saat', Interval.in_4_hour, 55))

    if count == 2:
        choice_2 = input("İkinci zaman dilimini seçin (1-6): ")
        interval_2 = intervals.get(choice_2, ('Günlük', Interval.in_daily, 20))
        return [interval_1, interval_2]

    return [interval_1]

# Trend Çizgisi Hesaplama Fonksiyonu
def calculate_trendlines(data, period=55):
    data['Trendline_High'] = np.nan
    data['Trendline_Low'] = np.nan

    for i in range(period, len(data)):
        high_prices = data['High'][i-period:i]
        low_prices = data['Low'][i-period:i]

        # Direnç Çizgisi (Yüksek Fiyatlar için Regresyon)
        x_high = np.arange(len(high_prices))
        slope_high, intercept_high, _, _, _ = linregress(x_high, high_prices)
        data.loc[data.index[i], 'Trendline_High'] = slope_high * len(x_high) + intercept_high

        # Destek Çizgisi (Düşük Fiyatlar için Regresyon)
        x_low = np.arange(len(low_prices))
        slope_low, intercept_low, _, _, _ = linregress(x_low, low_prices)
        data.loc[data.index[i], 'Trendline_Low'] = slope_low * len(x_low) + intercept_low

    return data

# Kırılım Kontrol Fonksiyonu
def trendline_breakout(data):
    data['Breakout_Up'] = data['Close'] > data['Trendline_High']
    data['Breakout_Down'] = data['Close'] < data['Trendline_Low']
    return data

# Loglama Fonksiyonu
def log_yaz(mesaj):
    print(mesaj)  # Dinamik bilgi mesajı

# TV Datafeed Bağlantısı
tv = TvDatafeed()

# Kripto Paralar
cryptos = ["1000SATSUSDT","1INCHUSDT","1MBABYDOGEUSDT","AAVEUSDT","ACAUSDT","ACEUSDT","ACHUSDT",
"ACMUSDT","ADAUSDT","ADXUSDT","AERGOUSDT","AEURUSDT","AEVOUSDT","AGLDUSDT","AIUSDT","AKROUSDT","ALCXUSDT",
"ALGOUSDT","ALICEUSDT","ALPACAUSDT","ALPHAUSDT","ALPINEUSDT","ALTUSDT","AMBUSDT","AMPUSDT","ANKRUSDT","APEUSDT",
"API3USDT","APTUSDT","ARBUSDT","ARDRUSDT","ARKMUSDT","ARKUSDT","ARPAUSDT","ARUSDT","ASRUSDT","ASTRUSDT","ASTUSDT",
"ATAUSDT","ATMUSDT","ATOMUSDT","AUCTIONUSDT","AUDIOUSDT","AVAUSDT","AVAXUSDT","AXLUSDT","AXSUSDT","BADGERUSDT","BAKEUSDT",
"BALUSDT","BANANAUSDT","BANDUSDT","BARUSDT","BATUSDT","BBUSDT","BCHUSDT","BEAMXUSDT","BELUSDT","BETAUSDT","BICOUSDT",
"BIFIUSDT","BLURUSDT","BLZUSDT","BNBUSDT","BNSOLUSDT","BNTUSDT","BNXUSDT","BOMEUSDT","BONKUSDT","BSWUSDT","BTCUSDT",
"BTTCUSDT","BURGERUSDT","C98USDT","CAKEUSDT","CATIUSDT","CELOUSDT","CELRUSDT","CFXUSDT","CHESSUSDT","CHRUSDT","CHZUSDT",
"CITYUSDT","CKBUSDT","CLVUSDT","COMBOUSDT","COMPUSDT","COSUSDT","COTIUSDT","CREAMUSDT","CRVUSDT","CTKUSDT","CTSIUSDT",
"CTXCUSDT","CVCUSDT","CVXUSDT","CYBERUSDT","DEXEUSDT","DFUSDT","DGBUSDT","DIAUSDT","DODOUSDT","DOGEUSDT","DOGSUSDT",
"DOTUSDT","DUSKUSDT","DYDXUSDT","DYMUSDT","EDUUSDT","EGLDUSDT","EIGENUSDT","ELFUSDT","ENAUSDT","ENJUSDT","ENSUSDT",
"EOSUSDT","ERNUSDT","ETCUSDT","ETHFIUSDT","ETHUSDT","EURIUSDT","EURUSDT","FARMUSDT","FDUSDTRY","FDUSDUSDT","FETUSDT",
"FIDAUSDT","FILUSDT","FIOUSDT","FIROUSDT","FISUSDT","FLMUSDT","FLOKIUSDT","FLOWUSDT","FLUXUSDT","FORTHUSDT","FTMUSDT",
"FTTUSDT","FUNUSDT","FXSUSDT","GALAUSDT","GASUSDT","GFTUSDT","GHSTUSDT","GLMRUSDT","GLMUSDT","GMTUSDT","GMXUSDT",
"GNOUSDT","GNSUSDT","GRTUSDT","GTCUSDT","GUSDT","HARDUSDT","HBARUSDT","HFTUSDT","HIFIUSDT","HIGHUSDT","HIVEUSDT",
"HMSTRUSDT","HOOKUSDT","HOTUSDT","ICPUSDT","ICXUSDT","IDEXUSDT","IDUSDT","ILVUSDT","IMXUSDT","INJUSDT","IOSTUSDT",
"IOTAUSDT","IOTXUSDT","IOUSDT","IQUSDT","IRISUSDT","JASMYUSDT","JOEUSDT","JSTUSDT","JTOUSDT","JUPUSDT","JUVUSDT",
"KAVAUSDT","KDAUSDT","KEYUSDT","KMDUSDT","KNCUSDT","KSMUSDT","LAZIOUSDT","LDOUSDT","LEVERUSDT","LINAUSDT","LINKUSDT",
"LISTAUSDT","LITUSDT","LOKAUSDT","LPTUSDT","LQTYUSDT","LRCUSDT","LSKUSDT","LTCUSDT","LTOUSDT","LUMIAUSDT","LUNAUSDT",
"LUNCUSDT","MAGICUSDT","MANAUSDT","MANTAUSDT","MASKUSDT","MAVUSDT","MBLUSDT","MBOXUSDT","MDTUSDT","MEMEUSDT","METISUSDT",
"MINAUSDT","MKRUSDT","MLNUSDT","MOVRUSDT","MTLUSDT","NEARUSDT","NEIROUSDT","NEOUSDT","NEXOUSDT","NFPUSDT","NKNUSDT","NMRUSDT",
"NOTUSDT","NTRNUSDT","NULSUSDT","OAXUSDT","OGNUSDT","OGUSDT","OMNIUSDT","OMUSDT","ONEUSDT","ONGUSDT","ONTUSDT","OPUSDT",
"ORDIUSDT","OSMOUSDT","OXTUSDT","PAXGUSDT","PDAUSDT","PENDLEUSDT","PEOPLEUSDT","PEPEUSDT","PERPUSDT","PHAUSDT","PHBUSDT","PIVXUSDT",
"PIXELUSDT","POLUSDT","POLYXUSDT","PONDUSDT","PORTALUSDT","PORTOUSDT","POWRUSDT","PROMUSDT","PROSUSDT","PSGUSDT","PUNDIXUSDT",
"PYRUSDT","PYTHUSDT","QIUSDT","QKCUSDT","QNTUSDT","QTUMUSDT","QUICKUSDT","RADUSDT","RAREUSDT","RAYUSDT","RDNTUSDT","REIUSDT",
"RENDERUSDT","RENUSDT","REQUSDT","REZUSDT","RIFUSDT","RLCUSDT","RONINUSDT","ROSEUSDT","RPLUSDT","RSRUSDT","RUNEUSDT","RVNUSDT",
"SAGAUSDT","SANDUSDT","SANTOSUSDT","SCRTUSDT","SCRUSDT","SCUSDT","SEIUSDT","SFPUSDT","SHIBUSDT","SKLUSDT","SLFUSDT","SLPUSDT",
"SNTUSDT","SNXUSDT","SOLUSDT","SPELLUSDT","SSVUSDT","STEEMUSDT","STGUSDT","STMXUSDT","STORJUSDT","STPTUSDT","STRAXUSDT","STRKUSDT",
"STXUSDT","SUIUSDT","SUNUSDT","SUPERUSDT","SUSHIUSDT","SXPUSDT","SYNUSDT","SYSUSDT","TAOUSDT","TFUELUSDT","THETAUSDT","TIAUSDT",
"TKOUSDT","TLMUSDT","TNSRUSDT","TONUSDT","TRBUSDT","TROYUSDT","TRUUSDT","TRXUSDT","TURBOUSDT","TUSDT","TUSDTRY","TUSDUSDT",
"TWTUSDT","UFTUSDT","UMAUSDT","UNIUSDT","USDCUSDT","USDPUSDT","USDTARS","USDTBRL","USDTCOP","USDTCZK","USDTDAI","USDTMXN",
"USDTPLN","USDTRON","USDTTRY","USDTUAH","USDTZAR","USTCUSDT","UTKUSDT","VANRYUSDT","VETUSDT","VIBUSDT","VICUSDT","VIDTUSDT",
"VITEUSDT","VOXELUSDT","VTHOUSDT","WANUSDT","WAXPUSDT","WBETHUSDT","WBTCUSDT","WIFUSDT","WINGUSDT","WINUSDT","WLDUSDT",
"WOOUSDT","WRXUSDT","WUSDT","XAIUSDT","XECUSDT","XLMUSDT","XNOUSDT","XRPUSDT","XTZUSDT","XVGUSDT","XVSUSDT","YFIUSDT","YGGUSDT",
"ZECUSDT","ZENUSDT","ZILUSDT","ZKUSDT","ZROUSDT","ZRXUSDT"]

# Kullanıcının seçtiği zaman dilimi
interval_count = get_interval_count_choice()
intervals = get_interval_choice(interval_count)
for interval_data in intervals:
    interval_name = interval_data[0]
    interval = interval_data[1]
    period = interval_data[2]  # Bar sayısı (period)
    print(f"Seçilen zaman dilimi: {interval_name}, Bar sayısı: {period}")

# Sonuçlar İçin DataFrame
df_trend_breakouts = pd.DataFrame(columns=['Kripto Para', 'Son Fiyat', 'Breakout Türü', 'Kırılım Zamanı', 'Zaman Dilimi'])

# Tarama işlemi
for symbol in tqdm(cryptos, desc="Tarama Yapılıyor", unit="kripto"):  # Progress bar ekleniyor
    for interval_data in intervals:
        try:
            interval_name = interval_data[0]
            interval = interval_data[1]
            period = interval_data[2]  # Bar sayısı (period)

            log_yaz(f"\n{symbol} için veri çekiliyor...")
            data = tv.get_hist(symbol=symbol, exchange='BINANCE', interval=interval, n_bars=500)
            if data is not None and not data.empty:
                log_yaz(f"{symbol}: Veri başarıyla çekildi. Satır sayısı: {len(data)}")
                data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)
                data = data.reset_index()

                # Trend Çizgilerini Hesapla
                data = calculate_trendlines(data, period=period)

                # Kırılımları Kontrol Et
                data = trendline_breakout(data)

                # Kırılım tespit edildi mi?
                if data['Breakout_Up'].iloc[-1]:
                    last_close = data['Close'].iloc[-1]
                    df_trend_breakouts.loc[len(df_trend_breakouts)] = [symbol, last_close, 'Yukarı Kırılım', data.index[-1], interval_name]
                    log_yaz(f"\033[33m{symbol}: Yukarı Kırılım tespit edildi. Fiyat: {last_close}\033[0m")
                elif data['Breakout_Down'].iloc[-1]:
                    last_close = data['Close'].iloc[-1]
                    df_trend_breakouts.loc[len(df_trend_breakouts)] = [symbol, last_close, 'Aşağı Kırılım', data.index[-1], interval_name]
                    log_yaz(f"\033[33m{symbol}: Aşağı Kırılım tespit edildi. Fiyat: {last_close}\033[0m")
                else:
                    log_yaz(f"{symbol}: Kırılım tespit edilmedi.")
            else:
                log_yaz(f"{symbol}: Veri bulunamadı.")
        except Exception as e:
            log_yaz(f"{symbol} için hata oluştu: {e}")

# Tarama sonucunda koşula uyanları ekrana yazdır
if not df_trend_breakouts.empty:
    print("\nKoşula Uyan Kripto Çiftleri Listesi:")
    pd.set_option('display.max_rows', None)  # Tüm satırlar
    pd.set_option('display.max_columns', None)  # Tüm sütunlar
    print(df_trend_breakouts)

    output_file = "kosula_uyan_kripto.xlsx"
    df_trend_breakouts.to_excel(output_file, index=False)
    print(f"\nKoşula Uyan Kripto Çiftleri Listesi '{output_file}' dosyasına kaydedildi.")
else:
    print("\nKoşula uyan bir durum tespit edilmedi.")


  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-6u5mjbh5
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-6u5mjbh5
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-ldyllyd4
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-ldyllyd4
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done


Kaç farklı zaman diliminde tarama yapmak istiyorsunuz?
1: Tek Zaman Dilimi
2: İki Zaman Dilimi
Seçiminizi yapın (1-2): 1
Lütfen zaman dilimi(lerini) seçin:
1: 15 Dakika - 144 bar
2: 30 Dakika - 89 bar
3: 1 Saat - 89 bar
4: 4 Saat - 89 bar
5: Günlük - 55 bar
6: Haftalık - 55 bar
Birinci zaman dilimini seçin (1-6): 1
Seçilen zaman dilimi: 15 Dakika, Bar sayısı: 144


Tarama Yapılıyor:   0%|          | 0/53 [00:00<?, ?kripto/s]


1000SATSUSDT için veri çekiliyor...
1000SATSUSDT: Veri başarıyla çekildi. Satır sayısı: 500


Tarama Yapılıyor:   2%|▏         | 1/53 [00:00<00:45,  1.15kripto/s]

...

DUSKUSDT: Yukarı kırılım tespit edildi. Fiyat: 0.2798

DYDXUSDT için veri çekiliyor...
DYDXUSDT: Veri başarıyla çekildi. Satır sayısı: 500


Tarama Yapılıyor:  98%|█████████▊| 52/53 [00:42<00:00,  1.38kripto/s]

DYDXUSDT: Yukarı kırılım tespit edildi. Fiyat: 1.7509

DYMUSDT için veri çekiliyor...
DYMUSDT: Veri başarıyla çekildi. Satır sayısı: 500


Tarama Yapılıyor: 100%|██████████| 53/53 [00:42<00:00,  1.23kripto/s]

DYMUSDT: Yukarı kırılım tespit edildi. Fiyat: 2.225

Koşula Uyan Kripto Çiftleri Listesi:
       Kripto Para   Son Fiyat   Breakout Türü  Kırılım Zamanı Zaman Dilimi
0     1000SATSUSDT    0.000267  Yukarı Kırılım             499    15 Dakika
1        1INCHUSDT    0.465000  Yukarı Kırılım             499    15 Dakika
2   1MBABYDOGEUSDT    0.002367   Aşağı Kırılım             499    15 Dakika
3         AAVEUSDT  231.650000  Yukarı Kırılım             499    15 Dakika
4          ACAUSDT    0.097000   Aşağı Kırılım             499    15 Dakika
5          ACHUSDT    0.029990  Yukarı Kırılım             499    15 Dakika
6          ACMUSDT    1.812000  Yukarı Kırılım             499    15 Dakika
7          ADXUSDT    0.209600   Aşağı Kırılım             499    15 Dakika
8         AEURUSDT    1.046500   Aşağı Kırılım             499    15 Dakika
9         AEVOUSDT    0.507300  Yukarı Kırılım             499    15 Dakika
10        AKROUSDT    0.004733   Aşağı Kırılım             499    15 Dakik